In [80]:
import sqlite3
from sqlite3 import Error
import pandas as pd
import re
import traceback
import time 


In [88]:
def create_connection(path):
    connection = None
    try:
        connection = sqlite3.connect(path)
        print("Connection to SQLite DB successful")
    except Error as e:
        print(f"Error'{e}' occurred")

    return connection

connection = create_connection(r"C:\Users\kyle.liu\DataGripProjects\Check_Com\save.sqlite")
ticks = time.time()


Connection to SQLite DB successful


In [78]:
'''
xl = pd.ExcelFile('C3321600.xlsx')
sheet_names = xl.sheet_names 
for i in sheet_names:
        input = xl.parse(i)
        input = input.drop_duplicates()
        input = input.replace('\n', '').replace(' ', '').replace('\t', '').replace('\r', '')
        input.to_sql(i, connection, if_exists='replace', index=False)
'''

In [87]:
cursor = connection.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table' ORDER by name")
tables_name = cursor.fetchall()
array1 = []
for tnames in tables_name:
    try:
        sql = "SELECT * FROM " + tnames[0]
        data_sql = pd.read_sql(sql,con = connection)
        input = pd.read_excel('C3321600.xlsx',tnames[0])#read excel
        input = input.drop_duplicates()
        input = input.replace('\n', '').replace(' ', '').replace('\t', '').replace('\r', '')
        output = pd.concat([input,data_sql],join = 'inner')
        input.to_sql(tnames[0], connection, if_exists='replace', index=False)
        addprimarykey1 = f'''create table {tnames[0]+str('_dg_tmp')} (ID   INTEGER constraint {tnames[0]+str('_pk')} primary key, 目标公司 TEXT,{tnames[0]} TEXT);'''
        addprimarykey2 = f'''insert into  {tnames[0]+str('_dg_tmp')}(ID, 目标公司, {tnames[0]}) select ID, 目标公司, {tnames[0]} from {tnames[0]};'''
        addprimarykey3 = f'''drop table {tnames[0]};'''
        addprimarykey4 = f'''alter table  {tnames[0]+str('_dg_tmp')} rename to {tnames[0]};'''
        cursor.execute(addprimarykey1)
        cursor.execute(addprimarykey2)
        cursor.execute(addprimarykey3)
        cursor.execute(addprimarykey4)
        connection.commit()
    except Exception as e:
        print(traceback.format_exc())
        error_doc = pd.DataFrame({"error": [e], "table": [tnames[0]],'time':[ticks]})
        error_doc.to_csv('error.csv', mode='a', header = False,index=False)
        print(f"Error '{e}' occurred,please check table{tnames[0]}")
        continue
